We import pandas to read the excel
And requests for API requests

In [1]:
import pandas as pd
import requests
import time

We read the workspace previously saved in a file in order to get back all the variables.

In [2]:
df = pd.read_pickle("./savedDF.pickle")

Or we load the file for the first execution

In [3]:
df = pd.read_excel('Data-sheet-STUDENTS.xlsx', header=2, index_col=0)

In [4]:
df["Country (according to CN Gov):"]

1                            Afghanistan
2                                Albania
3                                Armenia
4                             Azerbaijan
5                                Bahrain
6                             Bangladesh
7                                Belarus
8                                 Bhutan
9                 Bosnia and Herzegovina
10                                Brunei
11                              Bulgaria
12                              Cambodia
13                                 China
14                               Croatia
15                        Czech Republic
16                            East Timor
17                                 Egypt
18                               Estonia
19                      France (Not BRI)
20                               Georgia
21                     Germany (not BRI)
22                               Hungary
23                                 India
24                             Indonesia
25              

strip() clean begining and end of a string like "space" or "\n" ...

Next we get the isocode for each country

In [5]:
iso_codes = {}

In [6]:
for index, name in df["Country (according to CN Gov):"].items():    
    if name in iso_codes:
        continue
        
    clean_name = name.split(' (')[0]
    if name == "Thailiand":
        clean_name = "Thailand"
        
    if name == "The United Arab Emirates":
        clean_name = "United Arab Emirates"
        
    url = 'http://www.restcountries.eu/rest/v2/name/' + clean_name
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
        country_code = data[0]['alpha3Code']
        iso_codes[name] = country_code
    else:
        print('[ERROR]['+ name +'] we cannot get the data, reason is ', r.status_code, r.text)

In [7]:
iso_codes

{'Afghanistan': 'AFG',
 'Albania': 'ALB',
 'Armenia': 'ARM',
 'Azerbaijan': 'AZE',
 'Bahrain': 'BHR',
 'Bangladesh': 'BGD',
 'Belarus': 'BLR',
 'Bhutan': 'BTN',
 'Bosnia and Herzegovina': 'BIH',
 'Brunei': 'BRN',
 'Bulgaria': 'BGR',
 'Cambodia': 'KHM',
 'China': 'CHN',
 'Croatia': 'HRV',
 'Czech Republic': 'CZE',
 'East Timor': 'TLS',
 'Egypt': 'EGY',
 'Estonia': 'EST',
 'France (Not BRI)': 'FRA',
 'Georgia': 'GEO',
 'Germany (not BRI)': 'DEU',
 'Hungary': 'HUN',
 'India': 'IOT',
 'Indonesia': 'IDN',
 'Iran': 'IRN',
 'Iraq': 'IRQ',
 'Israel': 'ISR',
 'Jordan': 'JOR',
 'Kazakhstan': 'KAZ',
 'Kuwait': 'KWT',
 'Kyrgyzstan': 'KGZ',
 'Laos': 'LAO',
 'Latvia': 'LVA',
 'Lebanon': 'LBN',
 'Lithuania': 'LTU',
 'Macedonia': 'MKD',
 'Malaysia': 'MYS',
 'Maldives': 'MDV',
 'Moldova': 'MDA',
 'Mongolia': 'MNG',
 'Montenegro': 'MNE',
 'Myanmar': 'MMR',
 'Nepal': 'NPL',
 'Oman': 'OMN',
 'Pakistan': 'PAK',
 'Philippines': 'PHL',
 'Poland': 'POL',
 'Qatar': 'QAT',
 'Romania': 'ROU',
 'Russia': 'RUS',
 

We put back the iso_code into the dataframe

In [8]:
df["isocode"] = df["Country (according to CN Gov):"].apply(lambda x: iso_codes[x])

Next we want to get the indicator for each country

In [9]:
dicoColumnIndicator = {'IMF WB\nCustoms and other import duties (% of tax revenue) 2016': 'GC.TAX.IMPT.ZS', 'WB IMF Non-performing loans to total gross loans ratio (%)': 'FB.AST.NPER.ZS', 'UIC Rail freight (mn ton x km travelled)': 'IS.RRS.GOOD.MT.K6', 'ICAO air freight (mn ton x km travelled)': 'IS.RRS.GOOD.MT.K6', 'WB Household final consumption expenditure, PPP bn USD': 'NE.CON.PRVT.PP.CD', 'WB UNESCO R&D Expenditure % of GDP': 'GB.XPD.RSDV.GD.ZS', 'WB UNESCO Researchers p. mn inhabitants': 'SP.POP.SCIE.RD.P6', 'IFDI Net Inflows (% of GDP)': 'BX.KLT.DINV.WD.GD.ZS', 'WHO Current Health Expenditure p.c., PPP USD': 'SH.XPD.CHEX.PP.CD', 'WB Research Expenditure % of GDP': 'GB.XPD.RSDV.GD.ZS', 'WIPO Trademark applications': 'IP.TMK.RSCT', 'WIPO Patent Applications': 'IP.PAT.RESD'}

In [10]:
indicators = {}

In [11]:
for nameColumn, valueIndicator in dicoColumnIndicator.items():
    
    if nameColumn not in indicators:
        indicators[nameColumn] = {}
                
    for index, isocode in df["isocode"].items():    
        if isocode in indicators[nameColumn]:
            continue

        url = 'http://api.worldbank.org/v2/country/' + isocode + '/indicator/'+ valueIndicator +'?format=json&mrv=1'
        r = requests.get(url)
        if r.status_code == 200:
            print(isocode)
            data = r.json()        
            try:
                indicators[nameColumn][isocode] = data[1][0]['value']
                print(indicators[nameColumn][isocode])
            except:
                indicators[nameColumn][isocode] = "NA"
                print("NA")
        else:
            print('[ERROR]['+ isocode +'] we cannot get the data, reason is ', r.status_code, r.text)


AFG
35.203141190445
ALB
2.21645910864918
ARM
6.27539489392694
AZE
7.57184668387459
BHR
27.3860911270983
BGD
28.9230080455078
BLR
8.20061928730465
BTN
2.84050136682242
BIH
0.000155359665070619
BRN
NA
BGR
0.0688596099634017
KHM
13.542015604973
CHN
3.89434134127694
HRV
-0.0376305545169416
CZE
0.819566734190984
TLS
4.77991862435917
EGY
6.8490016570956
EST
0.039428987382724
FRA
-0.0232445343314451
GEO
0.757021008880042
DEU
0.0212485657218138
HUN
-0.000144686648107121
IOT
NA
IDN
2.56897518332382
IRN
20.6964502542622
IRQ
14.105211130071
ISR
0.962752553897723
JOR
7.00619344737872
KAZ
5.29951905118019
KWT
66.8067226890756
KGZ
17.6152097175161
LAO
NA
LVA
0.0755372366897075
LBN
6.04488095511726
LTU
0.711031058816326
MKD
4.65911362142491
MYS
1.56687280103591
MDV
17.845797817127
MDA
3.42323285739468
MNG
13.5021655958073
MNE
NA
MMR
9.00322150515809
NPL
17.7700383177162
OMN
NA
PAK
NA
PHL
20.3576079552952
POL
0.128545876239141
QAT
NA
ROU
0.035332354395594
RUS
6.21762623104324
SAU
23.0173047901435
SRB


Insert the data into the dataFrame

In [12]:
for nameColumn, indicator in indicators.items():
    df[nameColumn] = df["isocode"].apply(lambda x: indicator[x])

Creating the new excel with the data

In [13]:
df.to_excel('Data-sheet-STUDENTS-filled.xlsx')

We save the workspace to read it next time we continue the work and keep alive all the variable..

In [14]:
df.to_pickle("savedDF.pickle")

In [18]:
df

,Country (according to CN Gov):,Date of research inquiry,HF Index of Economic Freedom (Scale: 1-100),Heritage Foundation Index of Economic Freedom (Global Ranking),Bertelsmann Transformation index (BTI),Bertelsmann Transformation index (BTI) - Ranking,IMF WB Customs and other import duties (% of tax revenue) 2016,WB IMF Non-performing loans to total gross loans ratio (%),Moody's Socvereign Ratings List,S&P Sovereign Ratings List,...,WB Cost of Starting a Business,WB Ease of Doing Business Index (1 = most business-friendly regulations),WB Logistics Performance Index (LPI) (range: 1-5),IFDI Net Inflows (% of GDP),ED National Electric Power Consumption TwH,"WHO Current Health Expenditure p.c., PPP USD",WB Research Expenditure % of GDP,WIPO Trademark applications,WIPO Patent Applications,isocode
1,Afghanistan,NaT,NaN,NaN,NaN,NaN,35.2031,12.2018,NaN,NaN,...,NaN,NaN,NaN,0.718898,NaN,162.781,NA,NA,NA,AFG
2,Albania,NaT,NaN,NaN,NaN,NaN,2.21646,13.2325,NaN,NaN,...,NaN,NaN,NaN,8.01551,NaN,759.667,0.15412,1232,14,ALB
3,Armenia,NaT,NaN,NaN,NaN,NaN,6.27539,5.43232,NaN,NaN,...,NaN,NaN,NaN,2.04411,NaN,876.857,0.2277,2685,107,ARM
4,Azerbaijan,NaT,NaN,NaN,NaN,NaN,7.57185,NA,NaN,NaN,...,NaN,NaN,NaN,2.98895,NaN,1193.06,0.18521,2817,144,AZE
5,Bahrain,NaT,NaN,NaN,NaN,NaN,27.3861,NA,NaN,NaN,...,NaN,NaN,NaN,4.01407,NaN,1866.3,0.10116,253,8,BHR
6,Bangladesh,NaT,NaN,NaN,NaN,NaN,28.923,8.39791,NaN,NaN,...,NaN,NaN,NaN,1.07298,NaN,90.5984,NA,9247,61,BGD
7,Belarus,NaT,NaN,NaN,NaN,NaN,8.20062,12.8522,NaN,NaN,...,NaN,NaN,NaN,2.47207,NaN,1151.41,0.58716,4453,434,BLR
8,Bhutan,NaT,NaN,NaN,NaN,NaN,2.8405,8.41731,NaN,NaN,...,NaN,NaN,NaN,0.104527,NaN,293.11,NA,16,4,BTN
9,Bosnia and Herzegovina,NaT,NaN,NaN,NaN,NaN,0.00015536,10.0463,NaN,NaN,...,NaN,NaN,NaN,2.45426,NaN,1123.43,0.19951,742,87,BIH
10,Brunei,NaT,NaN,NaN,NaN,NaN,NA,3.5384,NaN,NaN,...,NaN,NaN,NaN,3.77013,NaN,1812.41,0.03701,176,8,BRN


In [16]:
df.columns

Index(['Country (according to CN Gov):', 'Date of research inquiry',
       'HF Index of Economic Freedom (Scale: 1-100)',
       'Heritage Foundation Index of Economic Freedom (Global Ranking)',
       'Bertelsmann Transformation index (BTI)',
       'Bertelsmann Transformation index (BTI) - Ranking',
       'IMF WB\nCustoms and other import duties (% of tax revenue) 2016',
       'WB IMF Non-performing loans to total gross loans ratio (%)',
       'Moody's Socvereign Ratings List', 'S&P Sovereign Ratings List',
       'Fitch Sovereign Ratings List',
       'UIC Rail freight (mn ton x km travelled)',
       'ICAO air freight (mn ton x km travelled)',
       'WB Household final consumption expenditure, PPP bn USD',
       'WB UNESCO R&D Expenditure % of GDP',
       'WB UNESCO Researchers p. mn inhabitants',
       'TI Corruption Perception Index',
       'TI Corruption Perception Index - Ranking', 'BRI GDP', 'BRI GDP p.c.',
       'BRI GDP growth rate', 'BRI Y.o.Y. GDP growth rate p.c

In [17]:
indicators

{'IMF WB\nCustoms and other import duties (% of tax revenue) 2016': {'AFG': 35.203141190445,
  'ALB': 2.21645910864918,
  'ARM': 6.27539489392694,
  'AZE': 7.57184668387459,
  'BHR': 27.3860911270983,
  'BGD': 28.9230080455078,
  'BLR': 8.20061928730465,
  'BTN': 2.84050136682242,
  'BIH': 0.000155359665070619,
  'BRN': 'NA',
  'BGR': 0.0688596099634017,
  'KHM': 13.542015604973,
  'CHN': 3.89434134127694,
  'HRV': -0.0376305545169416,
  'CZE': 0.819566734190984,
  'TLS': 4.77991862435917,
  'EGY': 6.8490016570956,
  'EST': 0.039428987382724,
  'FRA': -0.0232445343314451,
  'GEO': 0.757021008880042,
  'DEU': 0.0212485657218138,
  'HUN': -0.000144686648107121,
  'IOT': 'NA',
  'IDN': 2.56897518332382,
  'IRN': 20.6964502542622,
  'IRQ': 14.105211130071,
  'ISR': 0.962752553897723,
  'JOR': 7.00619344737872,
  'KAZ': 5.29951905118019,
  'KWT': 66.8067226890756,
  'KGZ': 17.6152097175161,
  'LAO': 'NA',
  'LVA': 0.0755372366897075,
  'LBN': 6.04488095511726,
  'LTU': 0.711031058816326,
  